<a href="https://colab.research.google.com/github/yuuki-kusumoto/kusumoto/blob/master/multi_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
!apt install aptitude swig

In [ ]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

In [ ]:
pip install MeCab

In [ ]:
pip install mecab-python3

In [ ]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git

In [ ]:
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [ ]:
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

In [ ]:
pip install neologdn

In [ ]:
pip install ipadic

In [ ]:
pip install mecab-python3

In [ ]:
pip install unidic-lite

In [3]:
# MeCabとtransformersを用意する
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# 以下で報告があるようにmecab-python3のバージョンを0.996.5にしないとtokezerで落ちる
# https://stackoverflow.com/questions/62860717/huggingface-for-japanese-tokenizer
!pip install mecab-python3==0.996.5
!pip install unidic-lite # これないとMeCab実行時にエラーで落ちる
!pip install transformers

     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 37.2MB/s 
     |████████████████████████████████| 901kB 42.1MB/s 


In [ ]:
pip install "transformers==2.5.1"

In [ ]:
import torch
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

In [ ]:
! curl http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/pn.csv.m3.120408.trim > pn.csv

In [ ]:
import transformers
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# model_nameはここから取得(cf. https://huggingface.co/transformers/pretrained_models.html)
model_name = "cl-tohoku/bert-base-japanese"
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

CSVデータの読み込み

In [ ]:
import pandas as pd

df1 = pd.read_csv('travel.csv', index_col=0)
df1.head()

In [ ]:
df1.columns=['text','breakfast_po',	'breakfast_ne',	'dinner_po',	'dinner_ne',	'bath_po',	'bath_ne',	'servis_po',	'service_ne',	'state_po',	'state_ne',	'facility_po',	'facility_ne'	,'room_po', 'room_ne']

In [ ]:
df = df1.fillna(0)
df.head()

In [ ]:
print(df.dtypes)

データフレーム内の値をfloat型からobject型に変換

In [ ]:
df_int2 = df[['breakfast_ne',	'dinner_po',	'dinner_ne',	'bath_po',	'bath_ne',	'servis_po',	'service_ne',	'state_po',	'state_ne',	'facility_po',	'facility_ne'	,'room_po', 'room_ne']].astype(int)
print(df_int2.dtypes)

In [ ]:
df_int2 = df[['breakfast_ne',	'dinner_po',	'dinner_ne',	'bath_po',	'bath_ne',	'servis_po',	'service_ne',	'state_po',	'state_ne',	'facility_po',	'facility_ne'	,'room_po', 'room_ne']].astype(int)
print(df_int2.dtypes)

In [ ]:
df_object = df_int2.astype(object)
print(df_object.dtypes)

In [ ]:
df_int1 = df[['text','breakfast_po']]
print(df_int1.dtypes)

In [ ]:
#データフレームを結合させる
df_concat = pd.concat([df_int1, df_object], axis =1)

In [ ]:
df = df_concat

In [ ]:
df2 = df[['text','dinner_po',	'dinner_ne',	'bath_po',	'bath_ne',	'servis_po',	'service_ne',	'state_po',	'state_ne',	'facility_po',	'facility_ne'	,'room_po', 'room_ne']]
df2 = df2.sample(frac=1, random_state=127).reset_index(drop=True)
df2.shape

In [ ]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df2) // 5

# 前から2割をテストデータとする
df2[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df2[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df2[len_0_2:].to_csv("./train.tsv", sep='\t', index=False, header=None)
print(df2[len_0_2:].shape)

In [ ]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  

In [ ]:
from torchtext.legacy import data
import torch
import torchtext  # torchtextを使用

In [ ]:
def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,include_lengths=True, batch_first=True, fix_length=512, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL2 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL3 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL4 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL5 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL6 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL7 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL8 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL9 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL10 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL11 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL12 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

In [ ]:
import torch
import torchtext
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer


# 日本語BERTの分かち書き用tokenizerを宣言
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
import transformers
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# model_nameはここから取得(cf. https://huggingface.co/transformers/pretrained_models.html)
model_name = "cl-tohoku/bert-base-japanese"
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

In [ ]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetに
# train_eval：61300個、test：15324個
dataset_train_eval, dataset_test = torchtext.legacy.data.TabularDataset.splits(
    path='.', train='train.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL), ('Label2', LABEL2), ('Label3', LABEL3),  ('Label4', LABEL4), ('Label5', LABEL5), ('Label6', LABEL6), ('Label7', LABEL7), ('Label8', LABEL8), ('Label9', LABEL9), ('Label10', LABEL10), ('Label11', LABEL11), ('Label12', LABEL12)])

In [ ]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：61300個、test：15324個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio = 1 - 15324 / 61300, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  

In [ ]:
print('訓練データの数', len(dataset_train_eval))
print('1つ目の訓練データkazu', vars(dataset_train_eval[10]))
print(vars(dataset_train_eval[0])['Label'])
print(len(vars(dataset_train_eval[0])))

# datasetの中身を文章に戻し、確認

print(tokenizer.convert_ids_to_tokens(item.Text.tolist()))  # 文章

In [ ]:
#@title
# 日本語BERTで扱える文章の長さは512
import seaborn as sns
title_length = df_concat2['text'].map(tokenizer.encode).map(len)
print(max(title_length))

sns.distplot(title_length)

In [ ]:
# DataLoaderの作成

batch_size = 12


dl_train = torchtext.legacy.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.legacy.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.legacy.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}

In [ ]:
# DataLoaderの動作確認 

batch = next(iter(dl_test))
print(batch)
print(batch.Text[0].shape)
print(batch.Label2.shape)

In [ ]:
from torch import nn


class BertForReview(nn.Module):
    '''BERTモデルにレビュー文の2クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForReview, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は12クラス
        self.cls = nn.Linear(in_features=768, out_features=12)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output

In [ ]:
#モデル構築
net = BertForReview()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')

In [ ]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True

In [ ]:
# 最適化手法の設定
import torch.optim as optim

# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 6e-6},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数
criterion = nn.BCEWithLogitsLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル
                labels2 = batch.Label2.to(device)
                labels3 = batch.Label3.to(device)
                labels4 = batch.Label4.to(device)
                labels5 = batch.Label5.to(device)
                labels6 = batch.Label6.to(device)
                labels7 = batch.Label7.to(device)
                labels8 = batch.Label8.to(device)
                labels9 = batch.Label9.to(device)
                labels10 = batch.Label10.to(device)
                labels11 = batch.Label11.to(device)
                labels12 = batch.Label12.to(device)
                pa = torch.stack([labels, labels2, labels3, labels4, labels5,labels6,labels7,labels8,labels9,labels10, labels11, labels12], dim = 1)

                pa = torch.tensor(pa, dtype=torch.float32)


                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)
##############################################################################################################################################
###################　　　　　　　ここまではおそらくあってる　　　　　　　###################################################################################
################################################################################################################################################

                    loss = criterion(outputs, pa)  # 損失を計算

#第二引数の1は行。axis = 1と同じ。
#torch.maxは最大値（テンソル）とその要素位置の２つを返しますが、その最大値を_で受けとっています。　ただ、最大値は不要なので適当な名前(_)の変数としています。
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時は逆誤差伝搬
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
#10イテレーションごとの計算で、割るときの分母にはバッチサイズに12をかけておく
                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == pa.data)
                                   ).double()/(12*batch_size)
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == pa.data)
                    

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [ ]:
# 学習・検証を実行する。
num_epochs = 1

net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)

In [ ]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

a = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

for batch in tqdm(dl_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル
    labels2 = batch.Label2.to(device)
    labels3 = batch.Label3.to(device)
    labels4 = batch.Label4.to(device)
    labels5 = batch.Label5.to(device)
    labels6 = batch.Label6.to(device)
    labels7 = batch.Label7.to(device)
    labels8 = batch.Label8.to(device)
    labels9 = batch.Label9.to(device)
    labels10 = batch.Label10.to(device)
    labels11 = batch.Label11.to(device)
    labels12 = batch.Label12.to(device)
    pa = torch.stack([labels, labels2, labels3, labels4, labels5,labels6,labels7,labels8,labels9,labels10, labels11, labels12], dim = 1)

    pa = torch.tensor(pa, dtype=torch.float32)



    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForReviewに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, pa)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == pa.data)  # 正解数の合計を更新
        
        cm = confusion_matrix(pa.cpu().numpy(), preds.cpu().numpy()) # 混同行列(numpy.ndarray)の取得
        print("コンフュージョンマトリックス")
        a = cm + a
        print(a)


# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))